In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
%config lnlineBackend.figure_format = 'retina'
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'# to avoid something wrong
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adam
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Flatten 
from keras import losses
from keras import regularizers
# prepare the libraries

In [ ]:
model = Sequential()
model.add(Conv2D(128,# number of kernels 
                kernel_size=(3, 3),activation='elu',input_shape=(150,150,1)))# the shape of the input should be (*,150,150,1)
model.add(MaxPooling2D(pool_size=(2, 2)))# the pooling layer (I tried average pooling but it seems to work worse)
model.add(Conv2D(64, (3, 3), activation='relu'))#the second convolutionary layer
model.add(MaxPooling2D(pool_size=(2, 2)))#extract the main features
model.add(Conv2D(64, (3, 3), activation='relu'))# the third convolutionary layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))#drop out to avoid overfitting
model.add(Flatten())# flattern transpose the high dimensional input into one dimension
model.add(Dense(128, activation='elu',kernel_regularizer=regularizers.l2(0.02)))# the first fully connected layer with regularization
# the activation is elu to help not to get stuck
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',#compile the model
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
model.load_weights('CNN_100_epochs_64_batch 0.6606 with augmentation.h5')

In [ ]:
def transform(result):# this function helps to get the otput of the model prediction into the final output
    y=[]
    for i in result:
        for j in (np.where(i==np.max(i))):# for each input data, select position the highest score(0,1,2,3)
            for m in j:#(it is in [],so we need to get it out)
                y.append(m+1)# thus it can be in the range of(1,4)
    resultfinal=np.array(y)
    return resultfinal

In [ ]:
X_t = np.load('./X_test.npy')
X_t = X_t.reshape(X_t.shape[0], 150, 150, 1)#reshape the test data
resultfinal=transform(model.predict(X_t))# change the output into only one value